<a href="https://colab.research.google.com/github/allyan/NL2SQL/blob/main/insert_dataset_to_weaviate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip3 install transformers
!pip3 install nltk
!pip3 install torch
!pip3 install weaviate-client==3.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import the BERT transformer model and pytorch

We are using the `bert-base-uncased` model in this example, but any model will work. Feel free to adjust accordingly.

## Initialize Weaviate Client
This assumes you have Weaviate running locally on `:8080`. Adjust URL accordingly. You could also enter the WCS URL here, for example, if you are running a WCS cloud instance instead of running Weaviate locally.

In [17]:
import torch
from transformers import AutoModel, AutoTokenizer
from nltk.tokenize import sent_tokenize
import weaviate

torch.set_grad_enabled(False)

# udpated to use different model if desired
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
model = AutoModel.from_pretrained(MODEL_NAME)
# model.to('cuda') # remove if working without GPUs
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# initialize nltk (for tokenizing sentences)
import nltk
nltk.download('punkt')

# initialize weaviate client for importing and searching
# client = weaviate.Client("http://64.71.146.93:8080")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
client = weaviate.Client("http://184.72.98.169:8080")

SCHEMA_NAME = "Salesforce"

## Load dataset from disk
Create some helper functions to create the dataset (20-newsgroup text posts) from disk. These methods are specific to the structure of your dataset, adjust accordingly.

In [19]:
import os
import random

questions = []
sqls = []

def read_dataset1():
    file_data = open("sample_data/salesforce_new_dataset.txt", 'r', encoding="utf8")
    for line in file_data:
        if line.split(" : ")[0] not in questions and line.split(" : ")[1] not in sqls:
            # print(line)
        # if line.split(" : ")[1] not in sqls:
            questions.append(line.split(" : ")[0])
            sqls.append(line.split(" : ")[1])

def read_dataset2():
    file_data = open("sample_data/custom_fields_dataset.txt", 'r', encoding="utf8")
    for line in file_data:
        if line.split(" : ")[0] not in questions and line.split(" : ")[1] not in sqls:
        # if line.split(" : ")[1] not in sqls:
            questions.append(line.split(" : ")[0])
            sqls.append(line.split(" : ")[1])

read_dataset1()
# read_dataset2()
print(len(questions))
       


1104


## Vectorize Dataset using BERT

The following is a helper function to vectorize all posts (using our BERT transformer) which are entered as an array. The return array contains all the vectors in the same order. BERT is optimized to run on GPUs, if you're using CPUs this might take a while. 

In [20]:
import time

def text2vec(text):
    tokens_pt = tokenizer(text, padding=True, truncation=True, max_length=500, add_special_tokens = True, return_tensors="pt")
    outputs = model(**tokens_pt)
    # tokens_pt.to('cuda') # remove if working without GPUs
    return outputs[0].mean(0).mean(0).detach()

def vectorize_questions(posts=[]):
    post_vectors=[]
    before=time.time()
    for i, post in enumerate(posts):
        vec=text2vec(sent_tokenize(post))
        post_vectors += [vec]
        if i % 500 == 0 and i != 0:
            print("So far {} objects vectorized in {}s".format(i, time.time()-before))
    after=time.time()
    
    print("Vectorized {} items in {}s".format(len(posts), after-before))
    
    return post_vectors

### Run everything we have so far

It is now time to run the functions we defined before. Let's load 50 random posts from disk, then vectorize them using BERT.

## Initialize Weaviate

Now that we have vectors we can import both the posts and the vectors into Weaviate, so we can then search through them.

### Init a simple schema
Our schema is very simple, we just have one object class, the "Post". A post class has just a single property, which we call "content" and is of type "text".

Each class in schema creates one index, so by running the below we tell weaviate to create one brand new vector index waiting for us to import data.

In [21]:
import json
schema = client.schema.get()
print(json.dumps(schema, indent=4))
class_obj = {
"class": SCHEMA_NAME, # <= Change to your class name - it will be your collection
"description": "SQL to Text mapping",
"vectorizer": "none",
"properties": [
    {
        "dataType": [
            "string"
        ],
        "description": "question",
        "name": "question"
    },
    {
        "dataType": [
            "string"
        ],
        "description": "sql",
        "name": "sql"
    }
]
}
client.schema.delete_class(SCHEMA_NAME)
client.schema.create_class(class_obj)

{
    "classes": [
        {
            "class": "Asana",
            "description": "SQL to Text mapping",
            "invertedIndexConfig": {
                "bm25": {
                    "b": 0.75,
                    "k1": 1.2
                },
                "cleanupIntervalSeconds": 60,
                "stopwords": {
                    "additions": null,
                    "preset": "en",
                    "removals": null
                }
            },
            "properties": [
                {
                    "dataType": [
                        "string"
                    ],
                    "description": "question",
                    "name": "question",
                    "tokenization": "word"
                },
                {
                    "dataType": [
                        "string"
                    ],
                    "description": "sql",
                    "name": "sql",
                    "tokenization": "word"
             

In [22]:

def import_questions_with_vectors(questions, sqls, vectors, client):
    if len(questions) != len(vectors):
        raise Exception("len of posts ({}) and vectors ({}) does not match".format(len(questions), len(vectors)))
        
    for i, question in enumerate(questions):
        try:
           client.data_object.create(
                data_object={"question": questions[i], "sql": sqls[i]},
                class_name=SCHEMA_NAME,
                vector=vectors[i]
            )
        except Exception as e:
            print(e)

In [23]:
def search(query="", limit=10):
    before = time.time()
    vec = text2vec(query)
    vec_took = time.time() - before

    before = time.time()
    near_vec = {"vector": vec}
    res = client \
        .query.get(SCHEMA_NAME, ["sql", "_additional {certainty}"]) \
        .with_near_vector(near_vec) \
        .with_additional("distance") \
        .with_limit(limit) \
        .do()
    search_took = time.time() - before

    # print("\nQuery \"{}\" with {} results took {:.3f}s ({:.3f}s to vectorize and {:.3f}s to search)" \
          # .format(query, limit, vec_took+search_took, vec_took, search_took))
    # print(res)
    for post in res["data"]["Get"][SCHEMA_NAME]:
        if post["_additional"]["distance"] < 0.8:
          print(post["sql"])
        # print('---')

In [ ]:
vectors = vectorize_questions(questions)

So far 500 objects vectorized in 48.19672894477844s


In [ ]:
import_questions_with_vectors(questions, sqls, vectors, client)

In [ ]:
search("show me active accounts ?", 10)


In [ ]:
#
